# Project 2: Topic Classification

In this project, you'll work with text data from newsgroup postings on a variety of topics. You'll train classifiers to distinguish between the topics based on the text of the posts. Whereas with digit classification, the input is relatively dense: a 28x28 matrix of pixels, many of which are non-zero, here we'll represent each document with a "bag-of-words" model. As you'll see, this makes the feature representation quite sparse -- only a few words of the total vocabulary are active in any given document. The bag-of-words assumption here is that the label depends only on the words; their order is not important.

The SK-learn documentation on feature extraction will prove useful:
http://scikit-learn.org/stable/modules/feature_extraction.html

Each problem can be addressed succinctly with the included packages -- please don't add any more. Grading will be based on writing clean, commented code, along with a few short answers.

As always, you're welcome to work on the project in groups and discuss ideas on the course wall, but please prepare your own write-up and write your own code.

In [1]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

Load the data, stripping out metadata so that we learn classifiers that only use textual features. By default, newsgroups data is split into train and test sets. We further split the test so we have a dev set. Note that we specify 4 categories to use for this project. If you remove the categories argument from the fetch function, you'll get all 20 categories.

In [2]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test',
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories)

num_test = len(newsgroups_test.target)
test_data, test_labels = newsgroups_test.data[num_test/2:], newsgroups_test.target[num_test/2:]
dev_data, dev_labels = newsgroups_test.data[:num_test/2], newsgroups_test.target[:num_test/2]
train_data, train_labels = newsgroups_train.data, newsgroups_train.target

print 'training label shape:', train_labels.shape
print 'test label shape:', test_labels.shape
print 'dev label shape:', dev_labels.shape
print 'labels names:', newsgroups_train.target_names

training label shape: (2034L,)
test label shape: (677L,)
dev label shape: (676L,)
labels names: ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']


(1) For each of the first 5 training examples, print the text of the message along with the label.

[2 pts]

In [7]:
### STUDENT START ###
def P1(num_examples=5):
    for i in range(num_examples):
        print '\n------------------------ training example #%d ' %(i+1) + '--------------------------------'
        print 'label number %d - label name: '  %(train_labels[i]) + newsgroups_train.target_names[train_labels[i]]
        print train_data[i]
    print '\n------------------------ End of P1 --------------------------------\n'
### STUDENT END ###

P1(5)



------------------------ training example #1 --------------------------------
label number 1 - label name: comp.graphics
Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych

------------------------ training example #2 --------------------------------
label number 3 - label name: talk.religion.misc


Seems to be, barring evidence to the contrary, that Koresh was simply
another deranged fanatic who thought it neccessary to take a whole bun

(2) Use CountVectorizer to turn the raw training text into feature vectors. You should use the fit_transform function, which makes 2 passes through the data: first it computes the vocabulary ("fit"), second it converts the raw text into feature vectors using the vocabulary ("transform").

The vectorizer has a lot of options. To get familiar with some of them, write code to answer these questions:

a. The output of the transform (also of fit_transform) is a sparse matrix: http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html. What is the size of the vocabulary? What is the average number of non-zero features per example? What fraction of the entries in the matrix are non-zero? Hint: use "nnz" and "shape" attributes.

b. What are the 0th and last feature strings (in alphabetical order)? Hint: use the vectorizer's get_feature_names function.

c. Specify your own vocabulary with 4 words: ["atheism", "graphics", "space", "religion"]. Confirm the training vectors are appropriately shaped. Now what's the average number of non-zero features per example?

d. Instead of extracting unigram word features, use "analyzer" and "ngram_range" to extract bigram and trigram character features. What size vocabulary does this yield?

e. Use the "min_df" argument to prune words that appear in fewer than 10 documents. What size vocabulary does this yield?

f. Using the standard CountVectorizer, what fraction of the words in the dev data are missing from the vocabulary? Hint: build a vocabulary for both train and dev and look at the size of the difference.

[6 pts]

In [4]:

### STUDENT START ###

def P2():
    print '\n------------------ Part a: vocabulary stats ------------------'
    uniVectorizer = CountVectorizer()
    docTermMatrix = uniVectorizer.fit_transform(train_data)    
    print 'size of the vocabulary: %d' %(docTermMatrix.shape[1])
    print 'average number of non-zero features per example: %2f' %(np.mean([doc.nnz for doc in docTermMatrix]))
    print 'fraction of the non-zero entries in the matrix: %2f' %(1.0*docTermMatrix.nnz/np.prod(docTermMatrix.shape))
    
    print '\n------------------ Part b: feature strings ------------------'
    featureNames = uniVectorizer.get_feature_names()
    print '0th feature string: ' + featureNames[0]
    print 'last feature string: ' + featureNames[-1]
    
    print '\n------------------ Part c: manual vocabulary ------------------'  
    # set 4-word vocabulary and re-fit
    uniVectorizer = CountVectorizer(vocabulary = ["atheism", "graphics", "space", "religion"])
    docTermMatrix = uniVectorizer.fit_transform(train_data)    
    # check shape of the new document-term matrix
    print 'new dimension: [%d x %d]' %(docTermMatrix.shape[0], docTermMatrix.shape[1])
    # average number of non-zero features per example
    print 'average number of non-zero features per example: %2f' %(np.mean([doc.nnz for doc in docTermMatrix]))
    
    print '\n------------------ Part d: bigram and trigram char feature ------------------'
    biVectorizer = CountVectorizer(analyzer='char', ngram_range=(2,2))    
    biVectorizer.fit(train_data)
    print 'size of vocabulary for bigram feature: %d' %len(biVectorizer.get_feature_names())    
    triVectorizer = CountVectorizer(analyzer='char', ngram_range=(3,3))
    triVectorizer.fit(train_data)
    print 'size of vocabulary for trigram feature: %d' %len(triVectorizer.get_feature_names())
    
    print '\n------------------ Part e: with min_df = 10 ------------------'
    uniVectorizer = CountVectorizer(min_df=10)
    uniVectorizer.fit(train_data)
    print 'size of vocabulary for unigram feature: %d' %(len(uniVectorizer.get_feature_names()))
        
    print '\n------------------ Part f: vocabulary discrepancy ------------------'
    vectorizer = CountVectorizer()
    vectorizer.fit(train_data)
    trainFeatures = vectorizer.get_feature_names()
    vectorizer.fit(dev_data)
    testFeatures = vectorizer.get_feature_names()
    commonFeatures = list(set(trainFeatures) & set(testFeatures))
    print 'Percentage of dev words missing in vocabulary: %2f' %(1.0*(len(testFeatures)-len(commonFeatures))/len(testFeatures))
    
#### STUDENT END ###

P2()


------------------ Part a: vocabulary stats ------------------
size of the vocabulary: 26879
average number of non-zero features per example: 96.705998
fraction of the non-zero entries in the matrix: 0.003598

------------------ Part b: feature strings ------------------
0th feature string: 00
last feature string: zyxel

------------------ Part c: manual vocabulary ------------------
new dimension: [2034 x 4]
average number of non-zero features per example: 0.268437

------------------ Part d: bigram and trigram char feature ------------------
size of vocabulary for bigram feature: 3291
size of vocabulary for trigram feature: 32187

------------------ Part e: with min_df = 10 ------------------
size of vocabulary for unigram feature: 3064

------------------ Part f: vocabulary discrepancy ------------------
Percentage of dev words missing in vocabulary: 0.247876


(3) Use the default CountVectorizer options and report the f1 score (use metrics.f1_score) for a k nearest neighbors classifier; find the optimal value for k. Also fit a Multinomial Naive Bayes model and find the optimal value for alpha. Finally, fit a logistic regression model and find the optimal value for the regularization strength C using l2 regularization. A few questions:

a. Why doesn't nearest neighbors work well for this problem?

b. Any ideas why logistic regression doesn't work as well as Naive Bayes?

c. Logistic regression estimates a weight vector for each class, which you can access with the coef\_ attribute. Output the sum of the squared weight values for each class for each setting of the C parameter. Briefly explain the relationship between the sum and the value of C.

[4 pts]

In [5]:
### STUDENT START ###
def P3():
    # use regular vectorizer to get features
    vectorizer = CountVectorizer()
    dtmTrain = vectorizer.fit_transform(train_data)
    dtmDev = vectorizer.transform(dev_data)
        
    # fit knn model
    kRange = range(5, 101, 5)
    print '\nknn - search k in ' + str(kRange)    
    clf = GridSearchCV(KNeighborsClassifier(), {'n_neighbors':kRange})
    clf.fit(dtmTrain, train_labels)    
    print 'knn best k: %d' %clf.best_params_['n_neighbors'] + ', with score: %f ' %clf.best_score_ 
    print 'knn f1 score: %f' %metrics.f1_score(dev_labels, clf.predict(dtmDev))
    
    # fit multinomial NB
    alphaRange = range(1,21,1)
    print '\nMultinomial NB - search alpha in ' + str(alphaRange)
    clf = GridSearchCV(MultinomialNB(), {'alpha':alphaRange})
    clf.fit(dtmTrain, train_labels)
    print 'Multinomial NB best alpha: %f' %clf.best_params_['alpha']     
    print 'Multinomial NB f1 score: %f ' %metrics.f1_score(dev_labels, clf.predict(dtmDev))
        
    # fit logistic regression
    cRange = range(1,10,1)
    print '\nLogistic regression - search C in ' + str(cRange)
    clf = GridSearchCV(LogisticRegression(penalty = 'l2'), {'C':cRange})
    clf.fit(dtmTrain, train_labels)
    print 'Logistic regression best C: %f' %clf.best_params_['C']
    print 'Logistic regression f1 score: %f' %metrics.f1_score(dev_labels, clf.predict(dtmDev))

### STUDENT END ###
P3()


knn - search k in [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]
knn best k: 95, with score: 0.442478 
knn f1 score: 0.453282

Multinomial NB - search alpha in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Multinomial NB best alpha: 1.000000
Multinomial NB f1 score: 0.777732 

Logistic regression - search C in [1, 2, 3, 4, 5, 6, 7, 8, 9]
Logistic regression best C: 1.000000
Logistic regression f1 score: 0.694417


ANSWER:
- a. on one hand the dimension of feature space is high, on the other hand the transformed feature vector is sparse, thus each sample is not very expressive in the feature space. This setup is not good for a distance-based clasifier like knn to perform well.   
- b.
- c.

(4) Train a logistic regression model. Find the 5 features with the largest weights for each label -- 20 features in total. Create a table with 20 rows and 4 columns that shows the weight for each of these features for each of the labels. Create the table again with bigram features. Any surprising features in this table?

[5 pts]

In [117]:
### STUDENT START ###
def EvaluateLogisticRegression(dtmTrain, featureNames, dtmDev):
    # fit logistic regression model    
    lrModel = LogisticRegression(C=1)
    lrModel.fit(dtmTrain, train_labels)
    nLargest = 5
    nLabel = lrModel.coef_.shape[0]
    weights = lrModel.coef_
    # get index for the largest weight for each label
    dtIndex = np.argpartition(np.abs(weights), -nLargest)[:,-nLargest:]    
    ind = set(np.reshape(dtIndex, (1, nLargest*nLabel))[0])
    # print the table    
    row_format ="{:>20}" * (nLabel + 1)
    print row_format.format("feature", *newsgroups_train.target_names)
    for i in ind:
        print row_format.format(featureNames[i], 
                                *[('* ' if i in dtIndex[j] else '') + '%4f' %weights[j,i] for j in range(nLabel)])
    # check f1 score
    print 'F1 score: %2f' %metrics.f1_score(dev_labels, lrModel.predict(dtmDev))

def P4():
    print '\nFeatures with maximum weights for each label - \'*\' denotes the weight that associates with the label'
    print 'Note: table may not have 20 rows since duplicate features are not printed'
    # use regular vectorizer to get features
    print '\nunigram word feature:'
    vectorizer = CountVectorizer()
    dtmTrain = vectorizer.fit_transform(train_data)
    EvaluateLogisticRegression(dtmTrain, vectorizer.get_feature_names(), vectorizer.transform(dev_data))
    
    # use strict bigram features - no single word
    print '\n(2,2) bigram word feature:'
    vectorizer = CountVectorizer(analyzer='word', ngram_range=(2,2))    
    dtmTrain = vectorizer.fit_transform(train_data)
    EvaluateLogisticRegression(dtmTrain, vectorizer.get_feature_names(), vectorizer.transform(dev_data))
    
     # use bigram features - allow single word
    print '\n(1,2) bigram word feature:'
    vectorizer = CountVectorizer(analyzer='word', ngram_range=(1,2))   
    dtmTrain = vectorizer.fit_transform(train_data)
    EvaluateLogisticRegression(dtmTrain, vectorizer.get_feature_names(), vectorizer.transform(dev_data))

### STUDENT END ###
P4()


Features with maximum weights for each label - '*' denotes the weight that associates with the label
Note: table may not have 20 rows since duplicate features are not printed

unigram word feature:
             feature         alt.atheism       comp.graphics           sci.space  talk.religion.misc
            graphics           -0.758229          * 1.937574         * -1.336043           -0.762691
            deletion          * 1.124913           -0.397898           -0.420370           -0.396143
               space         * -1.260505         * -1.316239          * 2.162220         * -1.170792
                file           -0.334778          * 1.266527           -0.806556           -0.626452
               blood           -0.533113           -0.106794           -0.316025          * 1.054753
              launch           -0.471136           -0.465556          * 0.936479           -0.332813
           christian           -0.607775           -0.418323           -0.270327          * 1.

ANSWER:
- For unigram feature: most words with maximum weights are closely related to the topic; but for strict bigram: some feature words (e.g. "is there", "and such", "was just", "looking for") are not reflecting the topic at all. 
- This is probably due to those feature words coincidentally appear in the very topic more often than the others, and those features can potentially bring a lot of noise and affect the performance negatively since they are not reflecting the truth - this is validated by the lower f1 score when using strict bigram word feature.
- When we allow single word in bigram feature, the best features are all from single word. This indicates that "bag of words" model works pretty well with this classifier, since the word context doesn't really help improve the performance.


(5) Try to improve the logistic regression classifier by passing a custom preprocessor to CountVectorizer. The preprocessing function runs on the raw text, before it is split into words by the tokenizer. Your preprocessor should try to normalize the input in various ways to improve generalization. For example, try lowercasing everything, replacing sequences of numbers with a single token, removing various other non-letter characters, and shortening long words. If you're not already familiar with regular expressions for manipulating strings, see https://docs.python.org/2/library/re.html, and re.sub() in particular. With your new preprocessor, how much did you reduce the size of the dictionary?

For reference, I was able to improve dev F1 by 2 points.

[4 pts]

In [7]:
def empty_preprocessor(s):
    return s

#def better_preprocessor(s):
### STUDENT START ###

### STUDENT END ###

#def P5():
### STUDENT START ###

    
### STUDENT END ###
#P5()

(6) The idea of regularization is to avoid learning very large weights (which are likely to fit the training data, but not generalize well) by adding a penalty to the total size of the learned weights. That is, logistic regression seeks the set of weights that minimizes errors in the training data AND has a small size. The default regularization, L2, computes this size as the sum of the squared weights (see P3, above). L1 regularization computes this size as the sum of the absolute values of the weights. The result is that whereas L2 regularization makes all the weights relatively small, L1 regularization drives lots of the weights to 0, effectively removing unimportant features.

Train a logistic regression model using a "l1" penalty. Output the number of learned weights that are not equal to zero. How does this compare to the number of non-zero weights you get with "l2"? Now, reduce the size of the vocabulary by keeping only those features that have at least one non-zero weight and retrain a model using "l2".

Make a plot showing accuracy of the re-trained model vs. the vocabulary size you get when pruning unused features by adjusting the C parameter.

Note: The gradient descent code that trains the logistic regression model sometimes has trouble converging with extreme settings of the C parameter. Relax the convergence criteria by setting tol=.01 (the default is .0001).

[4 pts]

In [8]:
def P6():
    # Keep this random seed here to make comparison easier.
    np.random.seed(0)

    ### STUDENT START ###

    

    ### STUDENT END ###
P6()

(7) Use the TfidfVectorizer -- how is this different from the CountVectorizer? Train a logistic regression model with C=100.

Make predictions on the dev data and show the top 3 documents where the ratio R is largest, where R is:

maximum predicted probability / predicted probability of the correct label

What kinds of mistakes is the model making? Suggest a way to address one particular issue that you see.

[4 pts]

In [11]:
#def P7():
    ### STUDENT START ###



    ### STUDENT END ###
#P7()

ANSWER:

(8) EXTRA CREDIT

Try implementing one of your ideas based on your error analysis. Use logistic regression as your underlying model.

- [1 pt] for a reasonable attempt
- [2 pts] for improved performance